In [2]:
#phase 2 
# step 1 defining the gcn layer
#here W is the learnable weights, this is the layer that can adjust while learning

import torch
import torch.nn as nn

In [3]:
class GCNLayer(nn.Module):
    #W can be created using nn.linear
    #infeatures: number of features per channel (right now, 1 — the EEG signal’s raw amplitude at that time step).
    #outfeatures: number of new features we want after mixing signals
    def __init__(self, in_features, out_features):
        # Always call the parent class's constructor
        super(GCNLayer, self).__init__()
        
        # W matrix is represented by this linear layer
        # It takes 'in_features' (e.g., 1 for raw signal)
        # and transforms them into 'out_features' (e.g., 64 new features per channel)
        self.linear = nn.Linear(in_features, out_features, bias=False)
        
    def forward(self, x, adj_matrix):
        #this is what the layer does on seeing the data 
        
        # Input 'x' shape: (batch_size, n_channels, in_features) e.g., (32, 62, 1)
        # Input 'adj_matrix' shape: (n_channels, n_channels) e.g., (62, 62)

        # Step 1: Spatial Aggregation (A_hat * X)
        # We use torch.bmm for batched matrix multiplication.
        # adj_matrix needs to be unsqueezed or expanded to match batch_size
        # The result will be (batch_size, n_channels, in_features)
        
        # Unsqueeze adj_matrix to (1, n_channels, n_channels) to enable broadcasting
        # across the batch dimension when multiplying with x.
        # Alternatively, you can explicitly expand it: adj_matrix.expand(x.size(0), -1, -1)
        
        # Note: In some GCN implementations, the order is X * W first, then A_hat * (XW).
        # Your description implies A_hat * X first, then * W. Let's follow that.
        
        # First, multiply adj_matrix with the feature matrix 'x' for each item in the batch
        # We need to make adj_matrix compatible with batch dimension for bmm
        # If x is (B, N, F_in), adj_matrix is (N, N)
        # We want (B, N, N) @ (B, N, F_in) -> (B, N, F_in)
        
        # This is a common way to handle it for batch processing with a shared A_hat:
        aggregated_features = torch.bmm(adj_matrix.unsqueeze(0).expand(x.size(0), -1, -1), x)
        
        # Step 2: Feature Transformation ( (A_hat * X) * W )
        # Apply the linear transformation (W matrix) to the aggregated features
        # self.linear operates on the last dimension (in_features -> out_features)
        transformed_features = self.linear(aggregated_features)
        
        # Step 3: Apply Activation Function
        output = torch.relu(transformed_features)
        
        return output

In [4]:
if __name__ == "__main__":
    print("--- Testing GCNLayer ---")

    # 1. Load your pre-computed A_hat (Adjacency Matrix)
    # Make sure 'adj_matrix.pt' is in the same directory or provide its full path
    try:
        adj_matrix_path = "/home/sanu/venvs/Ahat_adjacency_matrix.pt" # Adjust this path!
        A_hat = torch.load(adj_matrix_path)
        print(f"Loaded A_hat with shape: {A_hat.shape}")
    except FileNotFoundError:
        print(f"Error: Adjacency matrix file not found at {adj_matrix_path}.")
        print("Please ensure 'adj_matrix.pt' exists and the path is correct.")
        exit() # Exit if A_hat is not found

    # Define input and output features for the GCN layer
    in_features = 1  # Raw EEG signal has 1 feature per channel
    out_features = 64 # Chosen output feature dimension (hyperparameter)

    # 2. Instantiate the GCNLayer
    gcn_layer = GCNLayer(in_features=in_features, out_features=out_features)
    print(f"GCNLayer instantiated with in_features={in_features}, out_features={out_features}")

    # 3. Create a dummy EEG input X
    # Let's simulate a batch of 4 EEG samples, each with 62 channels and 1 feature per channel
    batch_size = 4
    n_channels = 62
    dummy_x = torch.randn(batch_size, n_channels, in_features) # Random values for testing
    print(f"Dummy input X shape: {dummy_x.shape}")

    # 4. Pass dummy data through the GCNLayer
    output = gcn_layer(dummy_x, A_hat)

    # 5. Print the shape of the output
    print(f"Output shape of GCNLayer: {output.shape}")

    # Expected output shape should be (batch_size, n_channels, out_features)
    # e.g., (4, 62, 64)
    expected_shape = (batch_size, n_channels, out_features)
    if output.shape == expected_shape:
        print("GCNLayer test passed: Output shape matches expected shape!")
    else:
        print(f"GCNLayer test FAILED: Expected {expected_shape}, got {output.shape}")

    print("--- GCNLayer Test Complete ---")

--- Testing GCNLayer ---
Loaded A_hat with shape: torch.Size([62, 62])
GCNLayer instantiated with in_features=1, out_features=64
Dummy input X shape: torch.Size([4, 62, 1])
Output shape of GCNLayer: torch.Size([4, 62, 64])
GCNLayer test passed: Output shape matches expected shape!
--- GCNLayer Test Complete ---


In [ ]:
# --- Main Simulation ---
if __name__ == "__main__":
    print("--- Simulating GCNLayer processing in STGTEncoder ---")

    # 1. Load your all_eeg_tensors (from eeg_tensors.pt)
    EEG_TENSORS_PATH = "/home/sanu/venvs/capstone/implementation/code/phase2/eeg_tensors.pt"
    try:
        all_eeg_data = torch.load(EEG_TENSORS_PATH)
        print(f"Loaded all_eeg_tensors from '{EEG_TENSORS_PATH}'. Shape: {all_eeg_data.shape}")
    except FileNotFoundError:
        print(f"Error: EEG tensors file not found at {EEG_TENSORS_PATH}.")
        print("Please ensure 'eeg_tensors.pt' exists at the specified path.")
        exit()
    except Exception as e:
        print(f"An error occurred loading EEG tensors: {e}")
        exit()

    # 2. Load your pre-computed A_hat (adjacency matrix)
    ADJ_MATRIX_PATH = "/home/sanu/Projects/Capstone_EEG/adj_matrix.pt" # Adjust this path!
    try:
        A_hat = torch.load(ADJ_MATRIX_PATH)
        print(f"Loaded A_hat with shape: {A_hat.shape}")
    except FileNotFoundError:
        print(f"Error: Adjacency matrix file not found at {ADJ_MATRIX_PATH}.")
        print("Please ensure 'adj_matrix.pt' exists at the specified path.")
        exit()
    except Exception as e:
        print(f"An error occurred loading A_hat: {e}")
        exit()

    # Define GCNLayer parameters
    in_features = 1       # Raw EEG signal has 1 feature per channel
    out_features = 64     # Output feature dimension of your GCNLayer (e.g., 64)
    n_channels = 62       # Number of EEG channels
    n_time_steps = 400    # Number of time steps in your EEG data

    # Instantiate your GCNLayer
    gcn_layer = GCNLayer(in_features=in_features, out_features=out_features)
    print(f"\nInstantiated GCNLayer with in_features={in_features}, out_features={out_features}")

    # --- Simulate a batch from a DataLoader ---
    # For demonstration, let's take a small batch of 4 samples from your all_eeg_data
    batch_size = 4
    if all_eeg_data.shape[0] < batch_size:
        print(f"Warning: Not enough samples ({all_eeg_data.shape[0]}) for batch size {batch_size}. Using all available samples.")
        batch_size = all_eeg_data.shape[0]

    # Select a batch of data (e.g., first 'batch_size' samples)
    eeg_batch_data = all_eeg_data[:batch_size, :, :] # Shape: (batch_size, 62, 400)
    print(f"Simulating a batch of EEG data with shape: {eeg_batch_data.shape}")

    # This list will store the output of the GCN for each time step
    # for all samples in the batch.
    gcn_outputs_per_timestep = []

    # --- Loop through each time step to process with GCNLayer ---
    print("\nProcessing each time step with GCNLayer...")
    for t in range(n_time_steps):
        # 1. Extract X for the current time step for ALL samples in the batch
        # This slice: eeg_batch_data[:, :, t] gives a tensor of shape (batch_size, 62)
        X_for_current_timestep_batch = eeg_batch_data[:, :, t]

        # 2. Reshape X to (batch_size, 62, 1) for the GCNLayer's input
        # The '1' is the 'in_features' dimension.
        X_for_gcn_input = X_for_current_timestep_batch.unsqueeze(2) 
        # print(f"  Time step {t}: X_for_gcn_input shape: {X_for_gcn_input.shape}") # Uncomment for verbose debugging

        # 3. Pass X and A_hat through the GCNLayer
        # Output will be (batch_size, n_channels, out_features)
        gcn_output_t = gcn_layer(X_for_gcn_input, A_hat)
        
        # 4. Store the output
        gcn_outputs_per_timestep.append(gcn_output_t)

    # --- After processing all time steps ---
    # Stack the list of outputs into a single tensor
    # gcn_outputs_per_timestep is a list of N_TIME_STEPS tensors, each (batch_size, 62, out_features)
    # torch.stack will combine them along a new dimension (dimension 0 by default, or specified)
    # We want (batch_size, N_TIME_STEPS, 62, out_features)
    final_gcn_sequence_output = torch.stack(gcn_outputs_per_timestep, dim=1)
    
    print("\nFinished processing all time steps with GCNLayer.")
    print(f"Shape of the final GCN sequence output: {final_gcn_sequence_output.shape}")

    # Expected output shape: (batch_size, n_time_steps, n_channels, out_features)
    expected_final_shape = (batch_size, n_time_steps, n_channels, out_features)
    if final_gcn_sequence_output.shape == expected_final_shape:
        print("Simulation successful: Final GCN sequence output shape matches expected!")
    else:
        print(f"Simulation FAILED: Expected {expected_final_shape}, got {final_gcn_sequence_output.shape}")

    print("\nThis `final_gcn_sequence_output` is what will typically be fed into your Temporal Transformer.")
    print("The next step in building the STGT Encoder is to integrate this into a full class.")
